In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# GENERAL CONSTANTS & WINDOWS
TREND_WINDOW = 6
SUSTAINED_WINDOW_MINUTES = 120
SUSTAINED_PERCENTAGE = 0.7

# UNIVERSAL THRESHOLDS (Keep your original)
THRESH_SPO2_LOW = 92
THRESH_SPO2_CRITICAL = 88
THRESH_SHOCK_INDEX_HIGH = 0.9
THRESH_TEMP_HIGH = 38.0

# AGE CATEGORY DEFINITIONS (Keep your original structure)
AGE_THRESHOLDS = {
    'infant': {
        'rr_low': 30,
        'rr_normal': 40,
        'rr_high': 60,
        'hr_low': 80,
        'hr_normal': 120,
        'hr_high': 160,
        'sbp_low': 70,
        'sbp_normal': 90,
        'sbp_high': 110,
        'temp_low': 35.5,
        'temp_normal': 36.8,
        'temp_high': 38.0
    },
    'child': {
        'rr_low': 14,
        'rr_normal': 20,
        'rr_high': 30,
        'hr_low': 70,
        'hr_normal': 90,
        'hr_high': 110,
        'sbp_low': 90,
        'sbp_normal': 105,
        'sbp_high': 120,
        'temp_low': 35.5,
        'temp_normal': 36.8,
        'temp_high': 38.0
    },
    'adult': {
        'rr_low': 12,
        'rr_normal': 16,
        'rr_high': 20,
        'hr_low': 60,
        'hr_normal': 80,
        'hr_high': 100,
        'sbp_low': 100,
        'sbp_normal': 115,
        'sbp_high': 130,
        'temp_low': 35.5,
        'temp_normal': 36.8,
        'temp_high': 38.0
    }
}

# ADRENAL CRISIS SPECIFIC THRESHOLDS (NEW)
THRESH_SBP_ABSOLUTE = 90
THRESH_SBP_CRITICAL = 80
THRESH_HR_TACHY = 100
THRESH_NA_LOW = 135
THRESH_K_HIGH = 5.0
THRESH_GLUCOSE_LOW = 70
THRESH_GLUCOSE_CRITICAL = 50

# SCORING WEIGHTS (NEW)
SCORE_WEIGHTS = {
    'SUSTAINED_HYPOTENSION': 3,
    'SUSTAINED_TACHYCARDIA': 1,
    'HYPONATREMIA': 2,
    'HYPERKALEMIA': 2,
    'HYPOGLYCEMIA': 3,
    'HISTORY_OF_AI': 4,
    'ON_STEROIDS': 3,
}

CRISIS_PROBABLE_THRESHOLD = 7
CRISIS_POSSIBLE_THRESHOLD = 4

In [3]:
def assign_age_category(df):
    """Assigns age category based on patient age."""
    df = df.copy()
    def _get_category(age):
        if age <= 1: return 'infant'
        elif age <= 12: return 'child'
        else: return 'adult'
    if 'age' in df.columns:
        df['age_category'] = df['age'].apply(_get_category)
    return df

def apply_vital_range_flags(df):
    """Applies comprehensive vital range flags."""
    df = df.copy()
    if 'age' in df.columns and 'age_category' not in df.columns:
        df = assign_age_category(df)
    if 'age_category' not in df.columns:
        df['age_category'] = 'adult'

    # Create a mapping series for efficient threshold application
    age_cats = df['age_category']
    thresh = AGE_THRESHOLDS

    # RESPIRATORY FLAGS
    if 'resp_rate' in df.columns:
        df['flag_rr_low'] = df['resp_rate'] < age_cats.map(lambda x: thresh[x]['rr_low'])
        df['flag_rr_high'] = df['resp_rate'] > age_cats.map(lambda x: thresh[x]['rr_high'])

    # SPO2 FLAGS
    if 'spo2' in df.columns:
        df['flag_spo2_low'] = df['spo2'] < THRESH_SPO2_LOW
        df['flag_spo2_critical'] = df['spo2'] < THRESH_SPO2_CRITICAL

    # TEMPERATURE FLAGS
    if 'temperature' in df.columns:
        df['flag_temp_low'] = df['temperature'] < age_cats.map(lambda x: thresh[x]['temp_low'])
        df['flag_temp_high'] = df['temperature'] > age_cats.map(lambda x: thresh[x]['temp_high'])

    # CARDIOVASCULAR FLAGS (General + Adrenal-specific)
    if 'heart_rate' in df.columns:
        df['flag_hr_low'] = df['heart_rate'] < age_cats.map(lambda x: thresh[x]['hr_low'])
        df['flag_hr_high'] = df['heart_rate'] > age_cats.map(lambda x: thresh[x]['hr_high'])
        df['flag_ac_hr_tachy'] = df['heart_rate'] >= THRESH_HR_TACHY  # Adrenal-specific

    if 'sbp' in df.columns:
        df['flag_sbp_low'] = df['sbp'] < age_cats.map(lambda x: thresh[x]['sbp_low'])
        df['flag_sbp_high'] = df['sbp'] > age_cats.map(lambda x: thresh[x]['sbp_high'])
        df['flag_ac_sbp_hypo'] = df['sbp'] < THRESH_SBP_ABSOLUTE  # Adrenal-specific
        df['flag_ac_sbp_critical'] = df['sbp'] < THRESH_SBP_CRITICAL  # Adrenal-specific

    # SHOCK INDEX
    if all(col in df.columns for col in ['heart_rate', 'sbp']):
        sbp_clipped = np.clip(df['sbp'], a_min=1, a_max=None)
        df['shock_index'] = df['heart_rate'] / sbp_clipped
        df['flag_shock_index_high'] = df['shock_index'] >= THRESH_SHOCK_INDEX_HIGH

    return df


In [4]:

def compute_recent_trends_delta(df):
    """
    Computes trends for each vital by differencing consecutive readings.
    Applies stricter interpretation using age-specific thresholds.
    """
    df = df.copy().sort_values("timestamp").reset_index(drop=True)

    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    trends = {}
    recent = df.tail(TREND_WINDOW)
    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'resp_rate': ('rr_low', 'rr_normal', 'rr_high'),
        'heart_rate': ('hr_low', 'hr_normal', 'hr_high'),
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)  # handled separately
    }

    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue

        y = recent[vital].dropna().values
        if len(y) < 2:
            continue

        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]
            high = thresholds[high_key]

            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        trends[f"{vital}_trend_flag"] = flag

    # Shock Index trend
    if all(col in recent.columns for col in ['heart_rate', 'sbp']):
        hr = recent['heart_rate'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp

        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)

            latest_si = si[-1]
            if latest_si >= THRESH_SHOCK_INDEX_HIGH:
                flag = "Shock Index high — improving" if avg_si_delta < 0 else "Shock Index high — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"

            trends['shock_index_trend_flag'] = flag

    return trends


In [5]:
def assess_adrenal_crisis_risk(vitals_df, lab_data=None, patient_context=None):
    """Main function to assess adrenal crisis risk."""
    df_with_flags = apply_vital_range_flags(vitals_df)
    trends = compute_recent_trends_delta(vitals_df)
    
    latest_time = df_with_flags['timestamp'].max()
    time_cutoff = latest_time - timedelta(minutes=SUSTAINED_WINDOW_MINUTES)
    recent_data = df_with_flags[df_with_flags['timestamp'] >= time_cutoff]
    
    sustained_sbp_hypo = recent_data['flag_ac_sbp_hypo'].mean() > SUSTAINED_PERCENTAGE
    sustained_hr_tachy = recent_data['flag_ac_hr_tachy'].mean() > SUSTAINED_PERCENTAGE
    
    # SAFETY OVERRIDES
    safety_override = False
    override_reason = []
    if recent_data['flag_ac_sbp_critical'].any():
        safety_override = True
        override_reason.append("SBP < 80 mmHg")
    if (lab_data is not None) and (lab_data.get('glucose', 999) < THRESH_GLUCOSE_CRITICAL):
        safety_override = True
        override_reason.append("Severe Hypoglycemia")
    
    if safety_override:
        return {
            "risk_classification": "CRITICAL",
            "score": None,
            "triggered_flags": ["SAFETY_OVERRIDE"],
            "override_reason": override_reason,
            "recommended_action": "IMMEDIATE ACTION: Call Rapid Response. Administer IV hydrocortisone immediately."
        }
    
    # SCORING ENGINE
    score = 0
    triggered_flags = []
    
    if sustained_sbp_hypo:
        score += SCORE_WEIGHTS['SUSTAINED_HYPOTENSION']
        triggered_flags.append("SUSTAINED_HYPOTENSION")
    if sustained_hr_tachy:
        score += SCORE_WEIGHTS['SUSTAINED_TACHYCARDIA']
        triggered_flags.append("SUSTAINED_TACHYCARDIA")
        
    if lab_data:
        if lab_data.get('sodium', 999) < THRESH_NA_LOW:
            score += SCORE_WEIGHTS['HYPONATREMIA']
            triggered_flags.append("HYPONATREMIA")
        if lab_data.get('potassium', 0) > THRESH_K_HIGH:
            score += SCORE_WEIGHTS['HYPERKALEMIA']
            triggered_flags.append("HYPERKALEMIA")
        if lab_data.get('glucose', 999) < THRESH_GLUCOSE_LOW:
            score += SCORE_WEIGHTS['HYPOGLYCEMIA']
            triggered_flags.append("HYPOGLYCEMIA")
            
    if patient_context:
        if patient_context.get('history_of_ai', False):
            score += SCORE_WEIGHTS['HISTORY_OF_AI']
            triggered_flags.append("HISTORY_OF_AI")
        if patient_context.get('on_steroids', False):
            score += SCORE_WEIGHTS['ON_STEROIDS']
            triggered_flags.append("ON_STEROIDS")
    
    # DECISION LOGIC
    if score >= CRISIS_PROBABLE_THRESHOLD:
        classification = "PROBABLE_ADRENAL_CRISIS"
        action = "URGENT: High likelihood of adrenal crisis. Treat empirically with steroids. Draw STAT cortisol, electrolytes, glucose."
    elif score >= CRISIS_POSSIBLE_THRESHOLD:
        classification = "POSSIBLE_ADRENAL_CRISIS"
        action = "INVESTIGATE: Suspicious pattern. Review history, order labs (Na, K, Glucose, Cortisol). Heighten monitoring."
    else:
        classification = "LOW_PROBABILITY"
        action = "Continue routine monitoring."
    
    sbp_trend_flag = trends.get('sbp_trend_flag', 'Unknown')
    if "worsening" in sbp_trend_flag and classification != "LOW_PROBABILITY":
        action += " Trend shows deterioration."
    
    return {
        "risk_classification": classification,
        "confidence_score": score,
        "triggered_flags": triggered_flags,
        "trend": sbp_trend_flag,
        "recommended_action": action
    }


In [6]:
if __name__ == "__main__":
    # Example data creation
    example_vitals = pd.DataFrame({
        'timestamp': pd.date_range(start='2023-10-27 10:00', periods=10, freq='15min'),
        'sbp': [95, 92, 88, 85, 82, 84, 83, 81, 85, 88],
        'heart_rate': [95, 98, 102, 105, 108, 110, 112, 115, 113, 110],
        'age': [45, 45, 45, 45, 45, 45, 45, 45, 45, 45]
    })
    
    example_labs = {'sodium': 132, 'potassium': 5.2, 'glucose': 85}
    example_context = {'history_of_ai': True, 'on_steroids': True}
    
    # Run the assessment
    result = assess_adrenal_crisis_risk(example_vitals, example_labs, example_context)
    print("Adrenal Crisis Assessment Result:")
    for key, value in result.items():
        print(f"{key}: {value}")

Adrenal Crisis Assessment Result:
risk_classification: PROBABLE_ADRENAL_CRISIS
confidence_score: 15
triggered_flags: ['SUSTAINED_HYPOTENSION', 'SUSTAINED_TACHYCARDIA', 'HYPONATREMIA', 'HYPERKALEMIA', 'HISTORY_OF_AI', 'ON_STEROIDS']
trend: Still abnormal — but improving
recommended_action: URGENT: High likelihood of adrenal crisis. Treat empirically with steroids. Draw STAT cortisol, electrolytes, glucose.
